In [1]:
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

In [2]:
df = pd.read_csv(r"C:\Users\Administrator.DAI-PC2\Desktop\ML\Day11\rfm_data_customer.csv", index_col =0)
df.head()

,revenue,most_recent_visit,number_of_orders,recency_days
customer_id,,,,
22086,777,2006-05-14,9,232
2290,1555,2006-09-08,16,115
26377,336,2006-11-19,5,43
24650,1189,2006-10-29,12,64
12883,1229,2006-12-09,12,23


In [3]:
df.drop('most_recent_visit', inplace=True, axis=1)
df

,revenue,number_of_orders,recency_days
customer_id,,,
22086,777,9,232
2290,1555,16,115
26377,336,5,43
24650,1189,12,64
12883,1229,12,23
...,...,...,...
3249,998,10,31
6686,771,8,187
16418,1016,9,154


In [4]:
# Create scaler: scaler
scaler = StandardScaler().set_output(transform='pandas')
dfscaled=scaler.fit_transform(df)

In [5]:
clust_DB = DBSCAN(eps=1, min_samples=2)
clust_DB.fit(dfscaled)
print(clust_DB.labels_)

[0 0 0 ... 0 0 0]


In [6]:
clust_df = df.copy()
clust_df["Clust"] = clust_DB.labels_
clust_df.sort_values(by='Clust')

,revenue,number_of_orders,recency_days,Clust
customer_id,,,,
22086,777,9,232,0
33150,1178,14,85,0
9942,1197,12,370,0
15845,1108,9,127,0
3918,1535,15,132,0
...,...,...,...,...
12225,907,13,89,0
29210,1174,12,65,0
23402,687,10,373,0


In [7]:
clust_df.groupby('Clust').mean()
clust_df.sort_values('Clust')

,revenue,number_of_orders,recency_days,Clust
customer_id,,,,
22086,777,9,232,0
33150,1178,14,85,0
9942,1197,12,370,0
15845,1108,9,127,0
3918,1535,15,132,0
...,...,...,...,...
12225,907,13,89,0
29210,1174,12,65,0
23402,687,10,373,0


In [8]:
dfscaled['Clust'] = clust_DB.labels_
df_scl_inliers = dfscaled[dfscaled['Clust']!=-1]
#print( silhouette_score(df_scl_inliers.iloc[:,:-1],
             #    df_scl_inliers.iloc[:,-1]) )

In [9]:
eps_range = [0.2,0.4,0.6,1]
mp_range = [2,3,4,5]
cnt = 0
a =[]
for i in eps_range:
    for j in mp_range:
        clust_DB = DBSCAN(eps=i, min_samples=j)
        clust_DB.fit(dfscaled.iloc[:,:5])
        if len(set(clust_DB.labels_)) > 2:
            cnt = cnt + 1
            dfscaled['Clust'] = clust_DB.labels_
            df_scl_inliers = dfscaled[dfscaled['Clust']!=-1]
            sil_sc = silhouette_score(df_scl_inliers.iloc[:,:-1],
                             df_scl_inliers.iloc[:,-1])
            a.append([cnt,i,j,sil_sc])
            print(i,j,sil_sc)
    
a = np.array(a)
pa = pd.DataFrame(a,columns=['Sr','eps','min_pt','sil'])
print("Best Paramters:")
pa[pa['sil'] == pa['sil'].max()]

0.2 2 -0.16754341558623703
0.2 3 -0.1294135723799745
0.2 4 -0.0968798265078369
0.2 5 -0.09069082307315968
0.4 2 -0.20134607323093318
0.4 3 -0.17215949766393684
0.4 4 -0.11646458109327724
0.4 5 -0.12237477965013906
0.6 2 -0.17811794642879175
0.6 3 -0.16561381464757327
0.6 4 -0.1314022474591124
0.6 5 -0.13113968199220233
1 2 -0.13726485545643954
1 3 -0.13726485545643954
1 4 -0.1372422232153952
1 5 -0.13716585492242267
Best Paramters:


,Sr,eps,min_pt,sil
3,4.0,0.2,5.0,-0.090691


In [10]:
clust_df = df.copy()
clust_df["Clust"] = clust_DB.labels_
clust_df.sort_values(by='Clust')

,revenue,number_of_orders,recency_days,Clust
customer_id,,,,
27148,1344,10,606,-1
23439,1490,10,18,-1
33599,2428,20,21,-1
3799,1440,21,65,-1
29050,983,6,12,-1
...,...,...,...,...
38514,1712,15,524,74
29734,1102,15,504,74
12967,1424,14,516,74


In [11]:
clust_df.groupby('Clust').mean()
clust_df.sort_values('Clust')

,revenue,number_of_orders,recency_days,Clust
customer_id,,,,
27148,1344,10,606,-1
23439,1490,10,18,-1
33599,2428,20,21,-1
3799,1440,21,65,-1
29050,983,6,12,-1
...,...,...,...,...
38514,1712,15,524,74
29734,1102,15,504,74
12967,1424,14,516,74


In [13]:
print(clust_df.groupby('Clust').value_counts())

Clust  revenue  number_of_orders  recency_days
-1     964      6                 33              1
       1114     7                 261             1
       1257     8                 69              1
       1344     10                606             1
       1440     21                65              1
                                                 ..
 74    1102     15                504             1
       1424     14                516             1
       1513     13                566             1
       1583     15                517             1
       1712     15                524             1
Length: 39633, dtype: int64


In [14]:
clust_DB.labels_

array([0, 1, 2, ..., 0, 9, 8], dtype=int64)